In [10]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pickle
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_SVM_RBF
from metrics import ClassificationResult
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
contacts='pe'
path='/users/mtaranov/datasets_3d/by_chr_dist_matched_'+contacts+'/'
day='d0'
thres='10'

X_train_node1 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'


X_test_node1 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_window = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

y_train = get_labels(path+day+'_y_train_thres_'+thres+'.npy')
y_test = get_labels(path+day+'_y_test_thres_'+thres+'.npy')
y_valid = get_labels(path+day+'_y_valid_thres_'+thres+'.npy')

X_train=concat_motifs([X_train_node1, X_train_node2])
X_test=concat_motifs([X_test_node1, X_test_node2])
X_valid=concat_motifs([X_valid_node1, X_valid_node2])

In [3]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)

In [4]:
X_train_pairs = X_train_normalized.reshape(X_train_normalized.shape[0],X_train_normalized.shape[2]*X_train_normalized.shape[3])
X_valid_pairs = X_valid_normalized.reshape(X_valid_normalized.shape[0],X_valid_normalized.shape[2]*X_valid_normalized.shape[3])
X_test_pairs = X_test_normalized.reshape(X_test_normalized.shape[0],X_test_normalized.shape[2]*X_test_normalized.shape[3])

In [5]:
X_train_pairs.shape

(6710, 3840)

# SVM RBF Predictions

In [6]:
best_param = pickle.load(open('/users/mtaranov/genome3D/examples/from_sequence_features/nodes_features/best_param_svm_rbf.pkl', 'rb' ) )

In [11]:
svm = Genome3D_SVM_RBF(best_param)

In [12]:
svm.train(X_train_pairs, y_train)
preds_test = svm.predict(X_test_pairs)
preds_train = svm.predict(X_train_pairs)
print ('On Test:\n{}\n'.format(ClassificationResult(y_test, preds_test)))
print ('On Train:\n{}\n'.format(ClassificationResult(y_train, preds_train)))

/users/mtaranov/local/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


On Test:
Balanced Accuracy: 61.64%	 auROC: 0.643	 auPRC: 0.605	 auPRG: 0.302
Positve Accuracy: 65.99%	 Negative Accuracy: 57.29%
Recall at 5%|10%|20% FDR: 0.0%|0.0%|0.0%	 Num Positives: 494	 Num Negatives: 494	 

On Train:
Balanced Accuracy: 100.00%	 auROC: 1.000	 auPRC: 1.000	 auPRG: 1.000
Positve Accuracy: 100.00%	 Negative Accuracy: 100.00%
Recall at 5%|10%|20% FDR: 100.0%|100.0%|100.0%	 Num Positives: 3355	 Num Negatives: 3355	 

